In [1]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow import keras

import numpy as np

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

TensorFlow version:  2.0.0


## Generating artificial data

In [3]:
data_size = 1000
# 80% of the data is for training.
train_pct = 0.8

train_size = int(data_size * train_pct)
print(train_size)

# Create some input data between -1 and 1 and randomize it.
x = np.linspace(-1, 1, data_size)
np.random.shuffle(x)

# Generate the output data.
# y = 0.5x + 2 + noise
y = 0.5 * x + 2 + np.random.normal(0, 0.05, (data_size, ))

# Split into test and train pairs.
x_train, y_train = x[:train_size], y[:train_size]
x_test, y_test = x[train_size:], y[train_size:]

800


## Training the model and logging loss

In [4]:
# First you need to define your logs directory and specify a tensorflow callback
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir = logdir) # pass the directory

model = keras.models.Sequential([
    keras.layers.Dense(16, input_dim = 1),
    keras.layers.Dense(1),
])

model.compile(
    loss = 'mse', # keras.losses.mean_squared_error
    optimizer = keras.optimizers.SGD(lr = 0.2),
)

print("Training ... With default parameters, this takes less than 10 seconds.")
training_history = model.fit(
    x_train, # input
    y_train, # output
    batch_size = train_size,
    verbose = 0, # Suppress chatty output; use Tensorboard instead
    epochs = 100,
    validation_data = (x_test, y_test),
    callbacks = [tensorboard_callback], # link it to the TF callback that we invoked before
)

print("Average test loss: ", np.average(training_history.history['loss']))

Training ... With default parameters, this takes less than 10 seconds.
Average test loss:  0.05677275192225352


## Using TensorBoard

In [5]:
# It's just sufficient to run the line below
%tensorboard --logdir logs/scalars